In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
import os


from google.colab import drive
drive.mount('/content/dgdrive')
from google.colab import files
PATH = '/content/dgdrive/Shared drives/InfoSci-Challenge5/'



population_training_df = pd.read_csv(PATH+'population_training.csv', encoding='cp1252').dropna(axis=0)
population_testing_df = pd.read_csv(PATH+'population_sample.csv', encoding='cp1252').dropna(axis=0)
population_testing_full_df = pd.read_csv(PATH+'population_testing.csv', encoding='cp1252').dropna(axis=0)
coef_df = pd.read_csv(PATH+'population_parameters_Cor.csv', index_col=0).dropna(axis=0)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/dgdrive


In [0]:
####autonomous correlation.

###creating the correlation matrix, 
FullCorrelatedMatrix = (population_training_df.drop(columns='Id'))
ind = (population_training_df.drop(columns='Id').columns)
FullCorrelatedMatrix = np.corrcoef(FullCorrelatedMatrix,rowvar=False)
FullCorrelatedMatrix = pd.DataFrame(FullCorrelatedMatrix, columns = ind, index=ind)



#creating a model for each country.
for country in FullCorrelatedMatrix:
  #creating the model.
  model = LinearRegression()

  #sorting the correlated countries in the matrix.
  CSCM = FullCorrelatedMatrix.loc[country]
  CSCM = CSCM.sort_values(ascending=False)

  #using a correlation matrix, finding the 4 most correlated countries.
  chosenFour = CSCM[:5].index.tolist()
  chosenFour = chosenFour

  #creating the matrix to properly parse & model & predict.
  # print(chosenFour)
  TrainingDF = population_training_df[chosenFour]
  #print(TrainingDF)
  xtrain = TrainingDF.drop(columns=country)
  ytrain = TrainingDF[country]
  
  #testing the model
  model.fit(xtrain,ytrain)
  xtest = population_testing_full_df[chosenFour].drop(columns=country)
  pred = model.predict(xtest)
  population_testing_df[country] = pred
  
  #creating a matrix w/ only the four most correlated countries, not including the country itself.
  onlyFour = chosenFour[1:]
  for i in range(len(onlyFour)):
    coef_df.loc[onlyFour[i],country] = float(model.coef_[i])

In [0]:
population_testing_df.to_csv('/content/dgdrive/Shared drives/InfoSci-Challenge5/population_prediction_Cor_v8_LinReg.csv', index = False)


In [0]:
coef_df.to_csv('/content/dgdrive/Shared drives/InfoSci-Challenge5/population_parameters_Cor_v1.csv')